In [569]:
import pandas as pd

def translate_team(team_name):
    # Check if translation exists, otherwise return the original name
    return translation_dict.get(team_name, f"Unknown ({team_name})")

Head_path = 'D:\Projects\Other_exp\Jupyter\orie4741-fp\prediction_data/'
# Step 1: Load the prediction CSV
prediction_df = pd.read_csv(Head_path+'prediction.csv')

# Step 2: Load the translation CSV (assuming it has 'Chinese' and 'English' columns)
translation_df = pd.read_csv(Head_path+'translation.csv', encoding='utf-8')

# Step 3: Create a dictionary for mapping Chinese team names to English
translation_dict = pd.Series(translation_df['English'].values, index=translation_df['Chinese'].values).to_dict()

# Step 3: Create a dictionary for mapping Chinese aHh to Math
translation_dict_math = pd.Series(translation_df['Math_Hh'].values, index=translation_df['Chinese_Hh'].values).to_dict()

# Step 4: Replace Chinese team names with English in HomeTeam and AwayTeam columns
prediction_df['HomeTeam'] = prediction_df['HomeTeam'].map(translation_dict).fillna(prediction_df['HomeTeam'])
prediction_df['AwayTeam'] = prediction_df['AwayTeam'].map(translation_dict).fillna(prediction_df['AwayTeam'])
prediction_df['AHh'] = prediction_df['AHh'].map(translation_dict_math).fillna(prediction_df['AHh'])
# prediction_df['HomeTeam'] = prediction_df['HomeTeam'].apply(translate_team)
# prediction_df['AwayTeam'] = prediction_df['AwayTeam'].apply(translate_team)

# Step 5: Save the updated dataframe (optional)
prediction_df.to_csv(Head_path+'updated_prediction.csv', index=False, encoding='utf-8-sig')
print(prediction_df)

    Unnamed: 0  Div       Date        HomeTeam       AwayTeam  B365H  B365D  \
0            1   E0  2025/1/26     Aston Villa       West Ham   1.53   4.33   
1            2  SP1  2025/1/26      Ath Bilbao        Leganes   1.38   5.00   
2            3  SP1  2025/1/26       Barcelona       Valencia   1.22   7.50   
3            4   E0  2025/1/26  Crystal Palace      Brentford   2.05   3.75   
4            5   E0  2025/1/26          Fulham     Man United   2.30   3.40   
5            6   D1  2025/1/26      Hoffenheim  Ein Frankfurt   3.10   3.60   
6            7   I1  2025/1/26           Lazio     Fiorentina   2.00   3.50   
7            8   F1  2025/1/26        Le Havre          Brest   3.30   3.10   
8            9   I1  2025/1/26           Lecce          Inter   9.50   5.25   
9           10   F1  2025/1/26            Lens         Angers   1.42   4.33   
10          11   I1  2025/1/26           Milan          Parma   1.30   6.00   
11          12   F1  2025/1/26          Nantes      

In [570]:
# import os
# 
# # 文件夹路径
# folder_path = 'D:\Projects\Other_exp\Jupyter\orie4741-fp\elo/'
# 
# # 获取文件夹下所有文件的名称
# file_names = []
# for root, dirs, files in os.walk(folder_path):
#     for file in files:
#         # 分离文件名和扩展名，获取文件名部分
#         file_name_without_extension = os.path.splitext(file)[0]
#         file_names.append(file_name_without_extension)
# 
# # 将文件名称写入到 txt 文件，按行存储
# with open(folder_path + 'file_names.txt', 'w', encoding='utf-8') as f:
#     for file_name in file_names:
#         f.write(file_name + '\n')
# 
# print("文件名称已成功保存到 file_names.txt")

In [571]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import joblib
from joblib import load, dump
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [572]:
df = pd.read_csv('prediction_data/updated_prediction.csv', low_memory = False)
df

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,...,Unnamed: 12,Unnamed: 13,truth_balance,truth,pre,pre_2,H,A,Unnamed: 20,Unnamed: 21
0,1,E0,2025/1/26,Aston Villa,West Ham,1.53,4.33,5.75,-1.00,1.92,...,NaN,NaN,-1,0,0,1,1,1,NaN,NaN
1,2,SP1,2025/1/26,Ath Bilbao,Leganes,1.38,5.00,7.50,-1.25,1.95,...,NaN,NaN,-1,0,0,0,0,0,NaN,1.0
2,3,SP1,2025/1/26,Barcelona,Valencia,1.22,7.50,10.00,-2.00,1.99,...,NaN,NaN,-2,2,2,0,7,1,NaN,NaN
3,4,E0,2025/1/26,Crystal Palace,Brentford,2.05,3.75,3.30,-0.50,2.07,...,NaN,NaN,-1,0,0,0,1,2,NaN,NaN
4,5,E0,2025/1/26,Fulham,Man United,2.30,3.40,3.10,-0.25,2.00,...,NaN,NaN,-1,0,0,0,0,1,NaN,NaN
5,6,D1,2025/1/26,Hoffenheim,Ein Frankfurt,3.10,3.60,2.20,0.25,1.92,...,NaN,NaN,1,2,2,1,2,2,NaN,NaN
6,7,I1,2025/1/26,Lazio,Fiorentina,2.00,3.50,3.80,-0.50,2.04,...,NaN,NaN,-1,0,0,1,1,2,NaN,NaN
7,8,F1,2025/1/26,Le Havre,Brest,3.30,3.10,2.35,0.25,1.87,...,NaN,NaN,1,1,0,0,0,1,NaN,1.0
8,9,I1,2025/1/26,Lecce,Inter,9.50,5.25,1.33,1.50,1.93,...,NaN,NaN,1,0,0,1,0,4,NaN,1.0
9,10,F1,2025/1/26,Lens,Angers,1.42,4.33,7.50,-1.25,2.03,...,NaN,NaN,-1,1,0,0,1,0,NaN,1.0


In [573]:
conditions = [
    df['AHh'] >= 2.5,         # AHh >= 2.75
    (df['AHh'] >= 1.5) & (df['AHh'] <= 2.25),  # 1.75 < AHh <= 2.75
    (df['AHh'] >= 0.25) & (df['AHh'] <= 1.25),  # 0.25 < AHh <= 1.75
    df['AHh'] == 0,            # AHh == 0
    (df['AHh'] >= -1.25) & (df['AHh'] <= -0.25),  # -1.75 < AHh <= -0.25
    (df['AHh'] >= -2.25) & (df['AHh'] <= -1.5),  # -2.75 < AHh <= -1.75
    df['AHh'] <= -2.5
]
labels = [3, 2, 1, 0, -1, -2, -3]

df['balance_val'] = np.select(conditions, labels)
df

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,...,Unnamed: 13,truth_balance,truth,pre,pre_2,H,A,Unnamed: 20,Unnamed: 21,balance_val
0,1,E0,2025/1/26,Aston Villa,West Ham,1.53,4.33,5.75,-1.00,1.92,...,NaN,-1,0,0,1,1,1,NaN,NaN,-1
1,2,SP1,2025/1/26,Ath Bilbao,Leganes,1.38,5.00,7.50,-1.25,1.95,...,NaN,-1,0,0,0,0,0,NaN,1.0,-1
2,3,SP1,2025/1/26,Barcelona,Valencia,1.22,7.50,10.00,-2.00,1.99,...,NaN,-2,2,2,0,7,1,NaN,NaN,-2
3,4,E0,2025/1/26,Crystal Palace,Brentford,2.05,3.75,3.30,-0.50,2.07,...,NaN,-1,0,0,0,1,2,NaN,NaN,-1
4,5,E0,2025/1/26,Fulham,Man United,2.30,3.40,3.10,-0.25,2.00,...,NaN,-1,0,0,0,0,1,NaN,NaN,-1
5,6,D1,2025/1/26,Hoffenheim,Ein Frankfurt,3.10,3.60,2.20,0.25,1.92,...,NaN,1,2,2,1,2,2,NaN,NaN,1
6,7,I1,2025/1/26,Lazio,Fiorentina,2.00,3.50,3.80,-0.50,2.04,...,NaN,-1,0,0,1,1,2,NaN,NaN,-1
7,8,F1,2025/1/26,Le Havre,Brest,3.30,3.10,2.35,0.25,1.87,...,NaN,1,1,0,0,0,1,NaN,1.0,1
8,9,I1,2025/1/26,Lecce,Inter,9.50,5.25,1.33,1.50,1.93,...,NaN,1,0,0,1,0,4,NaN,1.0,2
9,10,F1,2025/1/26,Lens,Angers,1.42,4.33,7.50,-1.25,2.03,...,NaN,-1,1,0,0,1,0,NaN,1.0,-1


In [574]:
# remove unknown columns
columns = df.columns
unknown_cols = [c for c in columns if "Unnamed" in c]
df = df.drop(columns=unknown_cols)

# remove rows with only na values
df = df.dropna(how="all")

features_to_keep = ["Div", "Date", "HomeTeam", "AwayTeam", "B365H", "B365D", "B365A","AHh", "B365AHH", "B365AHA", "balance_val"
                    ]
df = df[features_to_keep]

In [575]:
from os.path import exists
all_teams = set(df["HomeTeam"].unique()).union(set(df["AwayTeam"].unique()))

dfs = {}

for team in all_teams:
    if exists("elo/" + team + ".csv"):
        dfs[team] = pd.read_csv("elo/" + team + ".csv")
def find_team_elo_at(team_name, date):
    elo_df = dfs.get(team_name, None)
    if elo_df is None:
        return None
    elo_df["From"] = pd.to_datetime(elo_df["From"], format='mixed', dayfirst=True)
    elo_df["To"] = pd.to_datetime(elo_df["To"], format='mixed', dayfirst=True)
    if date > '2000-05-01':
        aa = 1
    # if date > '2024-10-01':
    #     aa = 2
    try:
        # elo = elo_df.loc[elo_df["From"] >= date]["Elo"].iloc[0]
        # print(elo_df.loc[(elo_df["From"] <= date) & (elo_df["To"] > date)])
        elo = elo_df.loc[(elo_df["From"] <= date) & (elo_df["To"] >= date)]["Elo"].iloc[0]
        return elo
    except:
        return None
    
def add_elo_feature(s):
    s["HomeTeamELO"] = find_team_elo_at(s["HomeTeam"], s["Date"])
    s["AwayTeamELO"] = find_team_elo_at(s["AwayTeam"], s["Date"])
    return s

f = np.vectorize(find_team_elo_at)

df["HomeTeamELO"] = f(df["HomeTeam"], df["Date"])
df["AwayTeamELO"] = f(df["AwayTeam"], df["Date"])

# df = df.apply(add_elo_feature, axis=1)
df.to_csv("prediction_data/updated_prediction_with_elo.csv")
df

,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,B365AHA,balance_val,HomeTeamELO,AwayTeamELO
0,E0,2025/1/26,Aston Villa,West Ham,1.53,4.33,5.75,-1.00,1.92,1.98,-1,1806.633667,1721.218262
1,SP1,2025/1/26,Ath Bilbao,Leganes,1.38,5.00,7.50,-1.25,1.95,1.95,-1,1785.690674,1583.090820
2,SP1,2025/1/26,Barcelona,Valencia,1.22,7.50,10.00,-2.00,1.99,1.91,-2,1888.577881,1617.742920
3,E0,2025/1/26,Crystal Palace,Brentford,2.05,3.75,3.30,-0.50,2.07,1.83,-1,1777.407593,1739.213623
4,E0,2025/1/26,Fulham,Man United,2.30,3.40,3.10,-0.25,2.00,1.90,-1,1765.261597,1755.285767
5,D1,2025/1/26,Hoffenheim,Ein Frankfurt,3.10,3.60,2.20,0.25,1.92,1.98,1,1599.335938,1740.221313
6,I1,2025/1/26,Lazio,Fiorentina,2.00,3.50,3.80,-0.50,2.04,1.89,-1,1826.267700,1765.172241
7,F1,2025/1/26,Le Havre,Brest,3.30,3.10,2.35,0.25,1.87,2.03,1,1507.878052,1691.590820
8,I1,2025/1/26,Lecce,Inter,9.50,5.25,1.33,1.50,1.93,2.00,2,1608.941162,1995.233765
9,F1,2025/1/26,Lens,Angers,1.42,4.33,7.50,-1.25,2.03,1.87,-1,1704.826782,1542.872192


In [576]:
df_noNa = df.dropna()
df_noNa.reset_index(inplace=True, drop=True)
df_noNa

,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,B365AHA,balance_val,HomeTeamELO,AwayTeamELO
0,E0,2025/1/26,Aston Villa,West Ham,1.53,4.33,5.75,-1.00,1.92,1.98,-1,1806.633667,1721.218262
1,SP1,2025/1/26,Ath Bilbao,Leganes,1.38,5.00,7.50,-1.25,1.95,1.95,-1,1785.690674,1583.090820
2,SP1,2025/1/26,Barcelona,Valencia,1.22,7.50,10.00,-2.00,1.99,1.91,-2,1888.577881,1617.742920
3,E0,2025/1/26,Crystal Palace,Brentford,2.05,3.75,3.30,-0.50,2.07,1.83,-1,1777.407593,1739.213623
4,E0,2025/1/26,Fulham,Man United,2.30,3.40,3.10,-0.25,2.00,1.90,-1,1765.261597,1755.285767
5,D1,2025/1/26,Hoffenheim,Ein Frankfurt,3.10,3.60,2.20,0.25,1.92,1.98,1,1599.335938,1740.221313
6,I1,2025/1/26,Lazio,Fiorentina,2.00,3.50,3.80,-0.50,2.04,1.89,-1,1826.267700,1765.172241
7,F1,2025/1/26,Le Havre,Brest,3.30,3.10,2.35,0.25,1.87,2.03,1,1507.878052,1691.590820
8,I1,2025/1/26,Lecce,Inter,9.50,5.25,1.33,1.50,1.93,2.00,2,1608.941162,1995.233765
9,F1,2025/1/26,Lens,Angers,1.42,4.33,7.50,-1.25,2.03,1.87,-1,1704.826782,1542.872192


In [577]:
# df_noNa_new = df_noNa.iloc[:3]
df_noNa_new = df_noNa

In [578]:
# 与历史数据合并，计算后续特征
history_df = pd.read_csv(Head_path+'history_data_balance.csv')
history_df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,WHA,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,easy_label,balance_val
0,F1,2003/8/2,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,3.1,5.00,-0.75,2.050,1.850,1702.604858,1611.196045,0,-1
1,F1,2003/8/2,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,3.1,2.40,0.00,1.925,1.975,1685.016113,1665.625732,0,0
2,D1,2003/8/2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,3.5,5.00,-0.75,1.800,2.100,1718.566284,1649.805298,0,-1
3,D1,2003/8/2,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,3.4,4.00,-0.75,2.025,1.875,1719.916748,1692.120972,0,-1
4,F1,2003/8/2,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,3.3,4.50,-0.75,1.900,2.000,1697.354004,1539.958130,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,I1,2025/1/26,Udinese,Roma,1.0,2.0,A,3.300,3.300,2.250,3.30,3.2,2.20,0.25,1.990,1.940,1667.065796,1772.048584,1,1
37342,SP1,2025/1/26,Vallecano,Girona,2.0,1.0,H,2.750,2.900,2.880,2.75,3.0,2.80,0.00,1.910,1.990,1635.277222,1704.093872,2,0
37343,SP1,2025/1/27,Alaves,Celta,1.0,1.0,D,2.630,3.000,3.000,2.62,3.2,2.75,0.00,1.890,2.010,1621.804321,1637.579956,1,0
37344,I1,2025/1/27,Genoa,Monza,2.0,0.0,H,2.250,3.000,3.600,2.25,3.0,3.40,-0.25,1.950,1.980,1682.297119,1628.680542,2,-1


In [579]:
# 合并两个 DataFrame，重置索引
df_noNa_comb = pd.concat([history_df[:-22], df_noNa_new], ignore_index=True)
df_noNa_comb

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,WHA,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,easy_label,balance_val
0,F1,2003/8/2,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,3.1,5.0,-0.75,2.050,1.850,1702.604858,1611.196045,0.0,-1
1,F1,2003/8/2,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,3.1,2.4,0.00,1.925,1.975,1685.016113,1665.625732,0.0,0
2,D1,2003/8/2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,3.5,5.0,-0.75,1.800,2.100,1718.566284,1649.805298,0.0,-1
3,D1,2003/8/2,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,3.4,4.0,-0.75,2.025,1.875,1719.916748,1692.120972,0.0,-1
4,F1,2003/8/2,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,3.3,4.5,-0.75,1.900,2.000,1697.354004,1539.958130,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,I1,2025/1/26,Udinese,Roma,NaN,NaN,NaN,3.300,3.300,2.250,NaN,NaN,NaN,0.25,1.990,1.940,1667.065796,1772.048584,NaN,1
37342,SP1,2025/1/26,Vallecano,Girona,NaN,NaN,NaN,2.750,2.900,2.880,NaN,NaN,NaN,0.00,1.910,1.990,1635.277222,1704.093872,NaN,0
37343,SP1,2025/1/27,Alaves,Celta,NaN,NaN,NaN,2.630,3.000,3.000,NaN,NaN,NaN,0.00,1.890,2.010,1621.804321,1637.579956,NaN,0
37344,I1,2025/1/27,Genoa,Monza,NaN,NaN,NaN,2.250,3.000,3.600,NaN,NaN,NaN,-0.25,1.950,1.980,1682.297119,1628.680542,NaN,-1


In [580]:
# # 加载编码器
# div_encoder = joblib.load('div_encoder.pkl')
# home_encoder = joblib.load('home_encoder.pkl')
# away_encoder = joblib.load('away_encoder.pkl')
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

div_encoder = OneHotEncoder()
home_encoder = OneHotEncoder()
away_encoder = OneHotEncoder()
# 你可以使用加载的编码器进行转换
onehot_div = div_encoder.fit_transform(df_noNa_comb.Div.values.reshape(-1,1)).toarray().astype(int)
onehot_div_df = pd.DataFrame(onehot_div, columns = ["Div "+str(int(i)) for i in range(onehot_div.shape[1])])

onehot_home = home_encoder.fit_transform(df_noNa_comb.HomeTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_home_df = pd.DataFrame(onehot_home, columns = ['HomeTeam ' + str(int(i)) for i in np.arange(onehot_home.shape[1])])

onehot_away = away_encoder.fit_transform(df_noNa_comb.AwayTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_away_df = pd.DataFrame(onehot_away, columns = ['AwayTeam ' + str(int(i)) for i in np.arange(onehot_away.shape[1])])

df_noNa_div = pd.concat([df_noNa_comb, onehot_div_df, onehot_home_df, onehot_away_df], axis = 1)
df_noNa_div.drop(columns = ['Div'], inplace = True)
df_noNa_div

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,...,AwayTeam 197,AwayTeam 198,AwayTeam 199,AwayTeam 200,AwayTeam 201,AwayTeam 202,AwayTeam 203,AwayTeam 204,AwayTeam 205,AwayTeam 206
0,2003/8/2,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,...,0,0,0,0,0,0,0,0,0,0
1,2003/8/2,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,...,0,0,0,0,0,0,0,0,0,0
2,2003/8/2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,...,0,0,0,0,0,0,0,0,0,0
3,2003/8/2,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,...,0,0,1,0,0,0,0,0,0,0
4,2003/8/2,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,2025/1/26,Udinese,Roma,NaN,NaN,NaN,3.300,3.300,2.250,NaN,...,0,0,0,0,0,0,0,0,0,0
37342,2025/1/26,Vallecano,Girona,NaN,NaN,NaN,2.750,2.900,2.880,NaN,...,0,0,0,0,0,0,0,0,0,0
37343,2025/1/27,Alaves,Celta,NaN,NaN,NaN,2.630,3.000,3.000,NaN,...,0,0,0,0,0,0,0,0,0,0
37344,2025/1/27,Genoa,Monza,NaN,NaN,NaN,2.250,3.000,3.600,NaN,...,0,0,0,0,0,0,0,0,0,0


In [581]:
target_encoder = LabelEncoder()
df_noNa_div['Result'] = target_encoder.fit_transform(df_noNa_div.easy_label) 

In [582]:
df_noNa_div['Year'] = pd.DatetimeIndex(df_noNa_div.Date).year

df_noNa_div['Month'] = pd.DatetimeIndex(df_noNa_div.Date).month
df_noNa_div['Sin_Month'] = np.sin(2*np.pi*df_noNa_div.Month/12)
df_noNa_div['Cos_Month'] = np.cos(2*np.pi*df_noNa_div.Month/12)

df_noNa_div['DayofYear'] = pd.DatetimeIndex(df_noNa_div.Date).dayofyear
df_noNa_div['Sin_Day'] = np.sin(2*np.pi*df_noNa_div.DayofYear/365)
df_noNa_div['Cos_Day'] = np.cos(2*np.pi*df_noNa_div.DayofYear/365)

# 注意 inplace是在原始frame修改，返回值是Nonetype
# learning_df = learning_df_div.drop(columns = ['Date','Month'], inplace = True)
df_noNa_div_date = df_noNa_div.drop(columns = ['Date','Month'])
df_noNa_div_date

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,...,AwayTeam 204,AwayTeam 205,AwayTeam 206,Result,Year,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day
0,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,3.1,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
1,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,3.1,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,3.5,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
3,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,3.4,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
4,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,3.3,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,Udinese,Roma,NaN,NaN,NaN,3.300,3.300,2.250,NaN,NaN,...,0,0,0,3,2025,0.500000,0.866025,26,0.432776,0.901502
37342,Vallecano,Girona,NaN,NaN,NaN,2.750,2.900,2.880,NaN,NaN,...,0,0,0,3,2025,0.500000,0.866025,26,0.432776,0.901502
37343,Alaves,Celta,NaN,NaN,NaN,2.630,3.000,3.000,NaN,NaN,...,0,0,0,3,2025,0.500000,0.866025,27,0.448229,0.893919
37344,Genoa,Monza,NaN,NaN,NaN,2.250,3.000,3.600,NaN,NaN,...,0,0,0,3,2025,0.500000,0.866025,27,0.448229,0.893919


In [583]:
# 定义一个函数来计算两队之间上一场比赛的结果
def compute_last_matches(df):
    
    unique_matchups = list(set((list(zip(df.HomeTeam, df.AwayTeam)))))
    df['Last Match Result'] = np.nan
    for home, away in unique_matchups:
        matchup_df = df[(df.HomeTeam == home) & (df.AwayTeam == away)]
        # 使用 shift(1) 方法将 FTR（全场比赛结果）列中的数据向下移动一行，这样每行的 last_match_result 将对应于这两队之前的一场比赛的结果。fill_value='Na' 确保了数据移动后空出的位置填充为 'Na'。
        # last_match_result = matchup_df.FTR.shift(1, fill_value='Na')
        last_match_result = matchup_df.Result.shift(1, fill_value=3)
        df.loc[matchup_df.index, 'Last Match Result'] = last_match_result
        
    lmr_encoder = LabelEncoder()
    df['Last Match Result'] = lmr_encoder.fit_transform(df['Last Match Result'])
    df.drop(columns = ['easy_label'], inplace = True)
    return df
df_noNa_div_date_lastMatch = compute_last_matches(df_noNa_div_date)
df_noNa_div_date_lastMatch

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,...,AwayTeam 205,AwayTeam 206,Result,Year,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day,Last Match Result
0,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,3.1,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
1,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,3.1,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,3.5,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
3,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,3.4,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
4,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,3.3,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,Udinese,Roma,NaN,NaN,NaN,3.300,3.300,2.250,NaN,NaN,...,0,0,3,2025,0.500000,0.866025,26,0.432776,0.901502,1
37342,Vallecano,Girona,NaN,NaN,NaN,2.750,2.900,2.880,NaN,NaN,...,0,0,3,2025,0.500000,0.866025,26,0.432776,0.901502,1
37343,Alaves,Celta,NaN,NaN,NaN,2.630,3.000,3.000,NaN,NaN,...,0,0,3,2025,0.500000,0.866025,27,0.448229,0.893919,2
37344,Genoa,Monza,NaN,NaN,NaN,2.250,3.000,3.600,NaN,NaN,...,0,0,3,2025,0.500000,0.866025,27,0.448229,0.893919,0


In [584]:
def compute_winstreak(df):
    
    years = df.Year.unique()
    df_lst = []    
    for year in years:
        
        year_df = df[df.Year == year]
        year_df['HomeWin'] = year_df.Result.replace([0, 1, 2], [0, 0, 1])
        year_df['AwayWin'] = year_df.Result.replace([0, 1, 2], [1, 0, 0])
        year_df['HomeWinStreak'] = None
        year_df['AwayWinStreak'] = None
        
        hometeams = year_df.HomeTeam.unique()
        awayteams = year_df.AwayTeam.unique()
        if year > 2024:
            # 将 AwayWin = 3 当作 0 来处理，保持计算连胜记录
            year_df['HomeWin'] = year_df['HomeWin'].replace(3, 1)
            year_df['AwayWin'] = year_df['AwayWin'].replace(3, 1)
        
        for team in hometeams:
            team_df = year_df[(year_df.HomeTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            team_grouper = (team_df.HomeWin != team_df.HomeWin.shift()).cumsum()
            team_df['HomeWinStreak'] = team_df[['HomeWin']].groupby(team_grouper).cumsum()
            team_df.loc[team_df.HomeWinStreak >0, 'HomeWinStreak'] -= 1
            year_df.loc[team_df.index, 'HomeWinStreak'] = team_df.HomeWinStreak
            
        for team in awayteams:
            # if team == 'Arsenal' and year == 2025:
            #     aa = 1
            team_df = year_df[(year_df.AwayTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            team_grouper = (team_df.AwayWin != team_df.AwayWin.shift()).cumsum()
            team_df['AwayWinStreak'] = team_df[['AwayWin']].groupby(team_grouper).cumsum()
            team_df.loc[team_df.AwayWinStreak >0, 'AwayWinStreak'] -= 1
            year_df.loc[team_df.index, 'AwayWinStreak'] = team_df.AwayWinStreak
            
        df_lst.append(year_df)
        
    return pd.concat(df_lst, axis = 0).drop(columns = ['HomeWin', 'AwayWin'])#,'DayofYear'])
df_noNa_div_date_lastMatch_streak = compute_winstreak(df_noNa_div_date_lastMatch)
df_noNa_div_date_lastMatch_streak

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,...,Result,Year,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak
0,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,3.1,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
1,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,3.1,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,3.5,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
3,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,3.4,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
4,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,3.3,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,Udinese,Roma,NaN,NaN,NaN,3.300,3.300,2.250,NaN,NaN,...,3,2025,0.500000,0.866025,26,0.432776,0.901502,1,1,0
37342,Vallecano,Girona,NaN,NaN,NaN,2.750,2.900,2.880,NaN,NaN,...,3,2025,0.500000,0.866025,26,0.432776,0.901502,1,1,1
37343,Alaves,Celta,NaN,NaN,NaN,2.630,3.000,3.000,NaN,NaN,...,3,2025,0.500000,0.866025,27,0.448229,0.893919,2,0,0
37344,Genoa,Monza,NaN,NaN,NaN,2.250,3.000,3.600,NaN,NaN,...,3,2025,0.500000,0.866025,27,0.448229,0.893919,0,0,0


In [585]:
def compute_winstodate(df):
    
    years = df.Year.unique()
    df_lst = []    
    for year in years:
        
        year_df = df[df.Year == year]
        year_df['HomeWin'] = year_df.Result.replace([0, 1, 2], [0, 0, 1])
        year_df['AwayWin'] = year_df.Result.replace([0, 1, 2], [1, 0, 0])
        year_df['HomeWinsToDate'] = None
        year_df['AwayWinsToDate'] = None
        
        hometeams = year_df.HomeTeam.unique()
        awayteams = year_df.AwayTeam.unique()
        if year > 2024:
            # 将 AwayWin = 3 当作 0 来处理，保持计算连胜记录
            year_df['HomeWin'] = year_df['HomeWin'].replace(3, 0)
            year_df['AwayWin'] = year_df['AwayWin'].replace(3, 0)
        for team in hometeams:
            team_df = year_df[(year_df.HomeTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

             # 计算截至当前场次之前的累计胜利次数（不包含当前场次）
            team_df['HomeWinsToDate'] = team_df.HomeWin.shift(1).cumsum()
            # 填充 NaN 值为 0，因为第一场比赛没有上一场比赛的数据
            team_df['HomeWinsToDate'].fillna(0, inplace=True)
            # 将更新后的数据回写到原 DataFrame
            year_df.loc[team_df.index, 'HomeWinsToDate'] = team_df.HomeWinsToDate
            
            # team_df['HomeWinsToDate'] = team_df.HomeWin.cumsum()
            # year_df.loc[team_df.index, 'HomeWinsToDate'] = team_df.HomeWinsToDate
            
        for team in awayteams:
            if team == 'Arsenal' and year == 2025:
                aa = 1
            team_df = year_df[(year_df.AwayTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))
            
            # 计算截至当前场次之前的累计胜利次数（不包含当前场次）
            team_df['AwayWinsToDate'] = team_df.AwayWin.shift(1).cumsum()
            # 填充 NaN 值为 0，因为第一场比赛没有上一场比赛的数据
            team_df['AwayWinsToDate'].fillna(0, inplace=True)
            # 将更新后的数据回写到原 DataFrame
            year_df.loc[team_df.index, 'AwayWinsToDate'] = team_df.AwayWinsToDate
            # team_df['AwayWinsToDate'] = team_df.AwayWin.cumsum()
            # year_df.loc[team_df.index, 'AwayWinsToDate'] = team_df.AwayWinsToDate
            
        df_lst.append(year_df)
        
    return pd.concat(df_lst, axis = 0).drop(columns = ['HomeWin', 'AwayWin','DayofYear'])
df_noNa_div_date_lastMatch_streak_windate = compute_winstodate(df_noNa_div_date_lastMatch_streak)
df_noNa_div_date_lastMatch_streak_windate.drop(columns = ['HomeTeam', 'AwayTeam'], inplace = True)
df_noNa_div_date_lastMatch_streak_windate

,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,WHA,AHh,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,1.0,2.0,A,1.727,3.100,4.500,1.66,3.1,5.0,-0.75,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
1,0.0,1.0,A,2.500,2.875,2.625,2.60,3.1,2.4,0.00,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
2,0.0,3.0,A,1.571,3.500,5.000,1.57,3.5,5.0,-0.75,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
3,0.0,3.0,A,1.833,3.200,3.750,1.72,3.4,4.0,-0.75,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
4,0.0,0.0,D,1.571,3.250,5.500,1.66,3.3,4.5,-0.75,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,NaN,NaN,NaN,3.300,3.300,2.250,NaN,NaN,NaN,0.25,...,2025,0.500000,0.866025,0.432776,0.901502,1,1,0,1.0,0.0
37342,NaN,NaN,NaN,2.750,2.900,2.880,NaN,NaN,NaN,0.00,...,2025,0.500000,0.866025,0.432776,0.901502,1,1,1,1.0,1.0
37343,NaN,NaN,NaN,2.630,3.000,3.000,NaN,NaN,NaN,0.00,...,2025,0.500000,0.866025,0.448229,0.893919,2,0,0,0.0,0.0
37344,NaN,NaN,NaN,2.250,3.000,3.600,NaN,NaN,NaN,-0.25,...,2025,0.500000,0.866025,0.448229,0.893919,0,0,0,0.0,0.0


In [586]:
betting_feats = ['B365H', 'B365D', 'B365A']
betting_feats

['B365H', 'B365D', 'B365A']

In [587]:
def compute_meanodds(df, betting_feats):
    """
    """
    home_odds = []
    away_odds = []
    draw_odds = []
    for odd in betting_feats:
        odd_type = odd[-1]
        if odd_type == 'H':
            home_odds.append(odd)
        elif odd_type == 'A':
            away_odds.append(odd)
        else:
            draw_odds.append(odd)
    avg_home_odds = df[home_odds].mean(axis=1)
    avg_away_odds = df[away_odds].mean(axis=1)
    avg_draw_odds = df[draw_odds].mean(axis=1)
    
    ordered_cols = ['HomeOdds', 'AwayOdds', 'DrawOdds'] + df.columns.tolist()
    
    df['HomeOdds'] = avg_home_odds
    df['AwayOdds'] = avg_away_odds
    df['DrawOdds'] = avg_draw_odds
    
    return df[ordered_cols]
df_odd = compute_meanodds(df_noNa_div_date_lastMatch_streak_windate, betting_feats)
df_odd.drop(columns = ['WHH', 'WHD', 'WHA', 'HomeOdds', 'AwayOdds', 'DrawOdds', 'FTHG', 'FTAG', 'FTR'], inplace = True)
df_odd

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,balance_val,Div 0,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,1.727,3.100,4.500,-0.75,2.050,1.850,1702.604858,1611.196045,-1,0,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
1,2.500,2.875,2.625,0.00,1.925,1.975,1685.016113,1665.625732,0,0,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
2,1.571,3.500,5.000,-0.75,1.800,2.100,1718.566284,1649.805298,-1,1,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
3,1.833,3.200,3.750,-0.75,2.025,1.875,1719.916748,1692.120972,-1,1,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
4,1.571,3.250,5.500,-0.75,1.900,2.000,1697.354004,1539.958130,-1,0,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,3.300,3.300,2.250,0.25,1.990,1.940,1667.065796,1772.048584,1,0,...,2025,0.500000,0.866025,0.432776,0.901502,1,1,0,1.0,0.0
37342,2.750,2.900,2.880,0.00,1.910,1.990,1635.277222,1704.093872,0,0,...,2025,0.500000,0.866025,0.432776,0.901502,1,1,1,1.0,1.0
37343,2.630,3.000,3.000,0.00,1.890,2.010,1621.804321,1637.579956,0,0,...,2025,0.500000,0.866025,0.448229,0.893919,2,0,0,0.0,0.0
37344,2.250,3.000,3.600,-0.25,1.950,1.980,1682.297119,1628.680542,-1,0,...,2025,0.500000,0.866025,0.448229,0.893919,0,0,0,0.0,0.0


In [588]:
from sklearn.preprocessing import MinMaxScaler
# 加载保存的 MinMaxScaler
# minmax_scaler = joblib.load('minmax_scaler_2025_01_30_22_23_33.pkl')
minmax_scaler = joblib.load('minmax_scaler_2025_01_31_00_44_24.pkl')
# minmax_scaler = MinMaxScaler()
df_odd.loc[:, ['Year']] = minmax_scaler.transform(df_odd.loc[:, ['Year']])

from sklearn.preprocessing import StandardScaler
# std_scaler = joblib.load('std_scaler2025_01_30_22_23_34.pkl')
std_scaler = joblib.load('std_scaler_2025_01_31_00_44_24.pkl')
# std_scaler = StandardScaler()
# to_scale = ['HomeWinStreak','AwayWinStreak','HomeWinsToDate', 'AwayWinsToDate', 'HomeTeamELO', 'AwayTeamELO', 'HomeOdds', 'AwayOdds', 'DrawOdds'] + betting_feats
to_scale = ['HomeTeamELO', 'AwayTeamELO'] + betting_feats
df_odd.loc[:, to_scale] = std_scaler.transform(df_odd.loc[:, to_scale])
df_odd

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,balance_val,Div 0,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,-0.516595,-0.677022,-0.031529,-0.75,2.050,1.850,0.123952,-0.652345,-1,0,...,-0.400000,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
1,-0.110737,-0.852421,-0.520079,0.00,1.925,1.975,-0.025470,-0.190534,0,0,...,-0.400000,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
2,-0.598502,-0.365203,0.098751,-0.75,1.800,2.100,0.259549,-0.324763,-1,1,...,-0.400000,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
3,-0.460941,-0.599068,-0.226949,-0.75,2.025,1.875,0.271021,0.034266,-1,1,...,-0.400000,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
4,-0.598502,-0.560090,0.229031,-0.75,1.900,2.000,0.079344,-1.256766,-1,0,...,-0.400000,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,0.309297,-0.521113,-0.617790,0.25,1.990,1.940,-0.177963,0.712415,1,0,...,1.066667,0.500000,0.866025,0.432776,0.901502,1,1,0,1.0,0.0
37342,0.020523,-0.832932,-0.453637,0.00,1.910,1.990,-0.448016,0.135851,0,0,...,1.066667,0.500000,0.866025,0.432776,0.901502,1,1,1,1.0,1.0
37343,-0.042482,-0.754977,-0.422369,0.00,1.890,2.010,-0.562472,-0.428489,0,0,...,1.066667,0.500000,0.866025,0.448229,0.893919,2,0,0,0.0,0.0
37344,-0.241998,-0.754977,-0.266033,-0.25,1.950,1.980,-0.048568,-0.503997,-1,0,...,1.066667,0.500000,0.866025,0.448229,0.893919,0,0,0,0.0,0.0


In [589]:
# 提取预测数据
testing_row_number = df_noNa_new.shape[0]
df_testing_data = df_odd.iloc[-testing_row_number:].loc[:, df_odd.columns != 'Result']
df_testing_data

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,balance_val,Div 0,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
37324,-0.620029,0.281823,0.294171,-1.00,1.92,1.98,1.007707,0.281143,-1,0,...,1.066667,0.5,0.866025,0.432776,0.901502,2,0,0,0.0,0.0
37325,-0.698785,0.804121,0.750152,-1.25,1.95,1.95,0.829790,-0.890805,-1,0,...,1.066667,0.5,0.866025,0.432776,0.901502,0,0,1,0.0,1.0
37326,-0.782792,2.752995,1.401553,-2.00,1.99,1.91,1.703847,-0.596798,-2,0,...,1.066667,0.5,0.866025,0.432776,0.901502,1,0,1,0.0,1.0
37327,-0.347006,-0.170315,-0.344201,-0.50,2.07,1.83,0.759423,0.433825,-1,0,...,1.066667,0.5,0.866025,0.432776,0.901502,2,1,1,1.0,1.0
37328,-0.215746,-0.443158,-0.396313,-0.25,2.00,1.90,0.656239,0.570190,-1,0,...,1.066667,0.5,0.866025,0.432776,0.901502,1,0,1,0.0,1.0
37329,0.204288,-0.287248,-0.630818,0.25,1.92,1.98,-0.753348,0.442375,1,1,...,1.066667,0.5,0.866025,0.432776,0.901502,0,0,0,0.0,0.0
37330,-0.373259,-0.365203,-0.213921,-0.50,2.04,1.89,1.174504,0.654072,-1,0,...,1.066667,0.5,0.866025,0.432776,0.901502,1,0,0,0.0,0.0
37331,0.309297,-0.677022,-0.591734,0.25,1.87,2.03,-1.530309,0.029768,1,0,...,1.066667,0.5,0.866025,0.432776,0.901502,0,0,1,0.0,1.0
37332,3.564560,0.999009,-0.857505,1.50,1.93,2.00,-0.671749,2.606039,2,0,...,1.066667,0.5,0.866025,0.432776,0.901502,0,0,0,0.0,0.0
37333,-0.677783,0.281823,0.750152,-1.25,2.03,1.87,0.142828,-1.232041,-1,0,...,1.066667,0.5,0.866025,0.432776,0.901502,2,0,1,0.0,1.0


In [590]:
### 加载模型
# model_name = 'sklearn_mlp_' + local_time + '.joblib'
# model_name = './' + 'sklearn_svm_2025_01_27_16_48_56.joblib'
# model_name = './' + 'sklearn_svm_randsearch_2025_01_27_17_09_20.joblib'
# model_name = './' + 'sklearn_mlp_2025_01_28_21_56_17.joblib'
model_name = './' + 'sklearn_mlp_2025_01_29_15_12_48.joblib'
mlp = load(model_name)
predict_val = mlp.predict(df_testing_data)
series_pre = pd.Series(predict_val, name='Predicted')
series_pre
# compare_result = pd.concat([series_pre, yTe.reset_index()], axis=1)
# compare_result

0     1
1     0
2     0
3     0
4     0
5     1
6     1
7     0
8     1
9     0
10    0
11    2
12    1
13    0
14    1
15    0
16    0
17    2
18    2
19    1
20    1
21    2
Name: Predicted, dtype: int64